# Campaign Contribution Data
- consolidating three files into one
- cleaning the data
- adding column for party ie: democrat, republican etc
- adding column for state

In [1]:
# Dependencies
import re
import pandas as pd
import numpy as np
import requests
import os
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from IPython.core.display import HTML
from datetime import date, datetime

In [2]:
contributions_2014 = pd.read_csv('campaign_contribution_2014.csv')
contributions_2016 = pd.read_csv('campaign_contribution_2016.csv')
contributions_2018 = pd.read_csv('campaign_contribution_2018.csv')
stateName = pd.read_csv('stateAbbrv.csv')

In [3]:
contributions_2014

,Representative,Office Running For,Total Raised
0,WarrenElizabeth Warren (D-Mass),Massachusetts Senate,$45964247
1,McConnellMitch McConnell (R-Ky)*,Kentucky Senate,$30845825
2,FrankenAl Franken (D-Minn)*,Minnesota Senate,$30770856
3,RubioMarco Rubio (R-Fla),Florida Senate,$30554421
4,GillibrandKirsten Gillibrand (D-NY),New York Senate,$30322285
...,...,...,...
95,SessionsJeff Sessions (R-Ala)*,Alabama Senate,$1712253
96,JohnsonTim Johnson (D-SD)*,South Dakota Senate,$1622563
97,RockefellerJay Rockefeller (D-WVa)*,West Virginia Senate,$1044452
98,JohannsMike Johanns (R-Neb)*,Nebraska Senate,$716208


In [4]:
stripList = contributions_2014['Representative']
namesNoAstrick = []
for i in stripList:
    namesNoAstrick.append(i[:-1])

In [5]:
#namesNoAstrick

In [6]:
def stripNames(stripList):
    state = []
    fullName = []
    party = []
    state = []
    for i in stripList:
        counter = 0
        for j in i:
            counter += 1
            if j == '(':
                party.append(i[counter:-1])
                state.append(i[counter+2:-1])
                fullName.append(i[:counter-2])
    return fullName, state, party

In [7]:
listName = stripNames(namesNoAstrick)

In [8]:
def cleanName(string):
    i = 1
    while i < len(string):
        if string[i] >= 'A' and string[i] <= 'Z':
            #print(i)
            return string[i:]
        else:
            i+= 1

In [9]:
#listName[0]

In [10]:
cleanedName = []
for i in listName[0]:
    cleanedName.append(cleanName(i))
cleanedName[:10]

['Elizabeth Warren',
 'ConnellMitch McConnell',
 'Al Franken',
 'Marco Rubio',
 'Kirsten Gillibrand',
 'Pat Toomey',
 'Charles E. Schumer',
 'Sherrod Brown',
 'Kay R. Hagan',
 'Harry Reid']

In [11]:
firstName = []
lastName = []
for i in cleanedName:
    j = 0
    while j < len(i):
        if i[j] == '.':
            #print(j)
            firstName.append(i[:j+1])
            lastName.append(i[j+2:])
            j += 1
        elif (i[j].isspace()) & ('.' not in i):
            firstName.append(i[:j])
            lastName.append(i[j:])
            j += 1
#         elif (i[j] == ' ') & (i[j+2] != '.'):
#             firstName.append(i[:j])
#             lastName.append(i[j:])
#             j += 1
        else:
            j+=1

In [12]:
party = []
fullPartyName = []
for i in listName[2]:
    if 'D' in i:
        party.append('D')
        fullPartyName.append('Democrat')
    elif 'R' in i:
        party.append('R')
        fullPartyName.append('Republican')
    else:
        party.append('I')
        fullPartyName.append('Independent/Other')

In [13]:
matchList = listName[1]
#matchList

In [14]:
AP = []
for i in stateName['Abbrev']:
    if i[-1] == '.':
        AP.append(i[:-1])
    else:
        AP.append(i)

In [15]:
stateName['AP'] = AP

In [16]:
abbrev = stateName['AP']
abbrev = abbrev.tolist()
#abbrev
fullState = (stateName['State']).tolist()
shortAbbrev = (stateName['Code']).tolist()
fullStateLower = []
for i in fullState:
    fullStateLower.append(i.lower())
stateDict = dict(zip(shortAbbrev, fullStateLower))

In [17]:
otherDict = dict(zip(AP, fullStateLower))
stateName['lower'] = fullStateLower

In [18]:
matchList[:10]

['Mass', 'Ky', 'Minn', 'Fla', 'NY', 'Pa', 'NY', 'Ohio', 'NC', 'Nev']

In [19]:
#stateDict

In [20]:
#Matches the strings that have AP style abbreviation with full statenames
def getMatch(string):
    for key, value in otherDict.items():
        if value.startswith(string):
            return value

In [21]:
#Calling getMatch with matchList input from csv
holder = []
for i in matchList:
    #print(i)
    i = i.lower()
    if (i == 'fla'):
        holder.append('florida')
    elif (i == 'wva'):
        holder.append('west virginia')
    elif len(i) > 2:
        holder.append(getMatch(i))
    elif len(i) == 2:
        holder.append(i.upper())
    else:
        print('none')
#holder

In [22]:
#Matches the full state names to abbreviations
def getMatch2(string):
    for key, value in stateDict.items():
        if value.startswith(string):
            return key

In [23]:
#Calling getMatch2 with holder from getMatch
renamedState = []
for i in holder:
    if len(i) == 2:
        renamedState.append(i)
    else:
        renamedState.append(getMatch2(i))


In [24]:
#stateDict

In [25]:
def longState(search_value):
    for key, val in stateDict.items():
        if key == search_value:
            return val

In [26]:
fullStateName = []
for i in renamedState:
    fullStateName.append((longState(i)).capitalize())
fullStateName[:5]

['Massachusetts', 'Kentucky', 'Minnesota', 'Florida', 'New york']

In [27]:
def upperTwo(string):
    if string.isspace():
        return True
    else:
        return False
    

In [28]:
def capitalTwo(check):
    holder = []
    i = 0
    while i < len(check):
        if upperTwo(check[i]):
            holder.append(' ')
            holder.append(check[i+1].capitalize())
            i += 1
        elif upperTwo(check[i-1]):
            i += 1
            continue
        else:
            holder.append(check[i])
            i += 1
    return ''.join(holder)

In [29]:
upperState = []
for i in fullStateName:
    upperState.append(capitalTwo(i))

In [30]:
#upperState

In [31]:
Total_Raised = []
for i in contributions_2014['Total Raised']:
    Total_Raised.append(int(i[1:]))


In [32]:
contributions_2014['Total_Raised'] = Total_Raised
contributions_2014['State_Abbrv'] = renamedState
contributions_2014['State'] = upperState
contributions_2014['Party'] = party
contributions_2014['Party_Name'] = fullPartyName
contributions_2014['Full_Name'] = cleanedName
contributions_2014['First_Name'] = firstName
contributions_2014['Last_Name'] = lastName

In [33]:
contributions_2014 = contributions_2014.drop(['Representative', 'Office Running For', 'Total Raised'], axis = 1)

In [34]:
contributions_2014 = contributions_2014[['First_Name', 'Last_Name', 'State', 'Party_Name', 'Total_Raised', 'Full_Name', 'State_Abbrv', 'Party',]]

In [35]:
contributions_2014['Full_Name'][1] = 'Mitch McConnell'
contributions_2014['Full_Name'][10] = 'John McCain'
contributions_2014['Full_Name'][11] = 'Claire McCaskill'

/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/Jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
contributions_2014= contributions_2014.sort_values('Last_Name').reset_index()
contributions_2014 = contributions_2014.drop(['index'], axis = 1)
contributions_2014.tail(60)

,First_Name,Last_Name,State,Party_Name,Total_Raised,Full_Name,State_Abbrv,Party
40,Johnny,Isakson,Georgia,Republican,9422559,Johnny Isakson,GA,R
41,Mike,Johanns,Nebraska,Republican,716208,Mike Johanns,NE,R
42,Ron,Johnson,Wisconsin,Republican,18631193,Ron Johnson,WI,R
43,Tim,Johnson,South Dakota,Democrat,1622563,Tim Johnson,SD,D
44,Tim,Kaine,Virginia,Democrat,19044379,Tim Kaine,VA,D
45,Angus,King,Maine,Independent/Other,3098168,Angus King,ME,I
46,Mark,Kirk,Illinois,Republican,17966562,Mark Kirk,IL,R
47,Amy,Klobuchar,Minnesota,Democrat,10070791,Amy Klobuchar,MN,D
48,Patrick,Leahy,Vermont,Democrat,4583423,Patrick Leahy,VT,D
49,Mike,Lee,Utah,Republican,3860376,Mike Lee,UT,R


In [41]:
newList = []
for i in contributions_2014['Last_Name']:
    if i[0].isspace():
        newList.append(i[1:])
    else:
        newList.append(i)
#print(len(newList))
contributions_2014 = contributions_2014.drop(['Last_Name'], axis = 1)
contributions_2014['Last_Name'] = newList
contributions_2014 = contributions_2014[['First_Name', 'Last_Name', 'State', 'Party_Name', 'Total_Raised', 'Full_Name', 'State_Abbrv', 'Party',]]

100


In [37]:
contributions_2014.to_csv('contributions_2014.csv', index=False)


In [38]:
!jupyter nbconvert --to python campaignContribution2014.ipynb

[NbConvertApp] Converting notebook campaignContribution2014.ipynb to python
[NbConvertApp] Writing 6503 bytes to campaignContribution2014.py
